In [1]:
from stockfish import Stockfish
import chess.pgn

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [24]:
# stockfish init
stockfish = Stockfish(
    '/home/vini/Personal/test_project/stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
    parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
)
stockfish.set_elo_rating(2600)
stockfish.set_skill_level(30)

In [67]:
# read game
def read_pgn(game_num=1):
    file_name = "../raw_data/Fics_data_pc_data.pgn"
#     with open(file_name) as pgn:
    pgn = open(file_name, encoding='UTF-8')
    
    
    game_counter = 0
    
    while True:
        try:
            game_counter += 1
            game = chess.pgn.read_game(pgn)

            board = game.board()
            moves = list(game.mainline_moves())
            variations = list(game.mainline())
            print(game.headers['FICSGamesDBGameNo'])
            new_game = chess.pgn.Game()
            new_pgn = open("new_game", "w", encoding="utf-8")

            new_game.headers["Event"] = "Example"
            new_game.headers["White"] = "Name One"
            new_game.headers["Black"] = "Name Two"
            new_game.headers["Event_two"] = "Example two"

            node = new_game.add_variation(moves[0])

            move_counter = 0
            for move in moves:
                move_counter += 1
                print(move)
                board.push(move)
                stockfish.set_fen_position(board.fen())
                eval = stockfish.get_evaluation()
                
#                 if move_counter == 1:
#                     node = new_game.add_variation(moves[0])
                node = node.add_variation(move)
                node.comment = str(eval['value'])


            if game_counter == game_num:
                break
            
        except AssertionError:
            break
#     print(new_game)
#     exporter = chess.pgn.FileExporter(new_pgn)
#     new_game.accept(exporter)

In [68]:
new_pc_game = read_pgn()

490373548
b2b3
e7e5
b1c3
b8c6
e2e3
g8f6
f1e2
d7d5
c1b2
f8d6
g1h3
a7a6
e1g1
h7h5
e2f3
e5e4
f3e2
c8h3
g2h3
d8d7
g1g2
e8c8
d2d4
h8h6
h3h4
h6g6
e2g4
f6g4
g2h1
g4h2
c3e4
d5e4
f1g1
d7h3
g1g3
g6g3
d1g1
h2f3
g1h2
h3h2


In [98]:
file_name = "../raw_data/Fics_data_pc_data.pgn"
pgn = open(file_name, encoding='UTF-8')
game = chess.pgn.read_game(pgn)
board = game.board()
moves = list(game.mainline_moves())


# print(node)
game_counter = 0
for move in moves:
    game_counter += 1
    stockfish.set_fen_position(board.fen())
    eval = stockfish.get_evaluation()
    
    if game_counter == 1:
        node = game.add_variation(moves[0])
        node.comment = str(eval['value'])
    else:
        node = node.add_variation(move)
        node.comment = str(eval)
    board.push(move)
    evaluations = str(game).split('(')[1].split(')')[0]
print(evaluations)

 1. b3 { 79 } 1... e5 { {'type': 'cp', 'value': -58 } 2. Nc3 { {'type': 'cp', 'value': 27 } 2... Nc6 { {'type': 'cp', 'value': -145 } 3. e3 { {'type': 'cp', 'value': 0 } 3... Nf6 { {'type': 'cp', 'value': -107 } 4. Be2 { {'type': 'cp', 'value': -20 } 4... d5 { {'type': 'cp', 'value': -130 } 5. Bb2 { {'type': 'cp', 'value': -54 } 5... Bd6 { {'type': 'cp', 'value': -193 } 6. Nh3 { {'type': 'cp', 'value': -9 } 6... a6 { {'type': 'cp', 'value': -227 } 7. O-O { {'type': 'cp', 'value': -208 } 7... h5 { {'type': 'cp', 'value': -303 } 8. Bf3 { {'type': 'cp', 'value': -157 } 8... e4 { {'type': 'cp', 'value': -1214 } 9. Be2 { {'type': 'cp', 'value': -349 } 9... Bxh3 { {'type': 'cp', 'value': -431 } 10. gxh3 { {'type': 'cp', 'value': -263 } 10... Qd7 { {'type': 'cp', 'value': -717 } 11. Kg2 { {'type': 'cp', 'value': -316 } 11... O-O-O { {'type': 'cp', 'value': -496 } 12. d4 { {'type': 'cp', 'value': -421 } 12... Rh6 { {'type': 'cp', 'value': -548 } 13. h4 { {'type': 'cp', 'value': -456 } 13... Rg

In [91]:
new_game = open("new_game", "w", encoding="utf-8")
exporter = chess.pgn.FileExporter(new_game)
game.accept(exporter)

3062

In [117]:
log_dict = {
    'cp':[],
    'move':[]
}

split_1 = evaluations.split("'value': ")
split_2 = [i.split('{')[0] for i in split_1]
# split_2
for j in split_2[1:]:
    if j.startswith('1. '):
        move = j
        cp = 0
    else:
        cp = j.split('}')[0]
        move = j.split('}')[1]
    print(move, cp)
# log_dict

 2. Nc3  -58 
 2... Nc6  27 
 3. e3  -145 
 3... Nf6  0 
 4. Be2  -107 
 4... d5  -20 
 5. Bb2  -130 
 5... Bd6  -54 
 6. Nh3  -193 
 6... a6  -9 
 7. O-O  -227 
 7... h5  -208 
 8. Bf3  -303 
 8... e4  -157 
 9. Be2  -1214 
 9... Bxh3  -349 
 10. gxh3  -431 
 10... Qd7  -263 
 11. Kg2  -717 
 11... O-O-O  -316 
 12. d4  -496 
 12... Rh6  -421 
 13. h4  -548 
 13... Rg6+  -456 
 14. Bg4  -625 
 14... Nxg4  -358 
 15. Kh1  -1493 
 15... Nxh2  -1157 
 16. Nxe4  -1487 
 16... dxe4  -1038 
 17. Rg1  -1584 
 17... Qh3  -884 
 18. Rg3  -1192 
 18... Rxg3  -2075 
 19. Qg1  -3 
 19... Nf3+  -2 
 20. Qh2  -2 
 20... Qxh2#  -1 
  -1 
